In [ ]:
def characters(f):
    """For f an open file, yield all its characters in order."""
    
    while True:
        c = f.read(1)
        if not c:
            return
        yield c

# Parsing WebSIS Text Files

In [ ]:
def allCharactersIn(checkForThese, inThis):
    """Check if all characters of one string are also in the other string."""
    
    for ch in inThis:
        if not ch in checkForThese:
            return False
    return True

In [ ]:
def parseWebsis(filename):
    """Read a single WebSIS student-enrollment record from the named file."""
    raise NotImplementedError

# Parsing XML Grades Data

In [ ]:
def parseXml(filename):
    """Parse one XML tag (with children) from a file."""
    with open(filename, "r") as f:
        # We maintain a stack of tags that we are still working on.
        # Each is an object that contains a list of children.
        # We start with a dummy tag standing for the document root.
        stack = [{"tag": "root", "children": [], "text": ""}]

        # Now we loop until there is no more content to read.
        while True:
            # Step 1: Find the start of the open tag.
            for ch in characters(f):
                if ch == '<':
                    # Oo, an open tag!  Exit the loop and start processing the tag name.
                    break
                else:
                    # This character has no special interpretation.  Add it to the string for the parent tag.
                    stack[-1]["text"] += ch

            # Step 2: Extract the tag name.
            is_closer = False
            tag = ""
            for ch in characters(f):
                if ch == '/':
                    # Oh, a closing tag.  We'll need to pop from the stack afterward.
                    is_closer = True
                elif ch == '>':
                    # Found the end
                    break
                else:
                    # Found another character in the tag name.
                    tag += ch

            # Step 3: Manipulate the tag stack appropriately.
            if tag == "":
                # We seem to have reached the end of the file.
                break
            elif is_closer:
                # For a closing tag, pop it off, adding it to the children list of the top of the stack,
                # after removing leading and trailing spaces.
                this_node = stack.pop()
                this_node["text"] = trim(this_node["text"])
                stack[-1]["children"].append(this_node)
            else:
                # For an opening tag, add it to the stack.
                stack.append({"tag": tag, "children": [], "text": ""})

        # Now we should find the main tag as the sole child of the root node.
        return stack[0]["children"][0]

In [ ]:
grades = parseXml("grades.xml")
grades

In [ ]:
def assignmentFromXml(xml):
    """Convert one assignment (pset or quiz) to a nicer format."""
    
    number = [int(child["text"])
              for child in xml["children"]
              if child["tag"] == "number"][0]

    students = {email["text"]: int(grade["text"])
                for child in xml["children"]
                if child["tag"] == "student"
                for email in child["children"]
                if email["tag"] == "email"
                for grade in child["children"]
                if grade["tag"] == "grade"}

    return {"number": number, "students": students}

def categoryFromXml(xml, singular):
    """Convert one category (psets or quizzes) to a nicer format."""
    
    raise NotImplementedError

def gradesFromXml(xml):
    """Convert a whole grades database from XML to a nicer format."""
    
    psets = [child
             for child in xml["children"]
             if child["tag"] == "psets"][0]
    quizzes = [child
               for child in xml["children"]
               if child["tag"] == "quizzes"][0]

    return {"psets": categoryFromXml(psets, "pset"),
            "quizzes": categoryFromXml(quizzes, "quiz")}

Bringing it all together:

In [ ]:
def parseGrades(filename):
    """Return the nice version of the grades database found in the file."""
    
    return gradesFromXml(parseXml(filename))

After all that fuss implementing an XML parser, we could have just used one that comes with Python.  Here's an example showing how.

In [ ]:
import xml.etree.ElementTree as ET

def assignmentFromXmlEasier(xml):
    number = int(xml.find("number").text)
    students = {child.find("email").text: int(child.find("grade").text)
                for child in xml.findall("student")}

    return {"number": number, "students": students}

def categoryFromXmlEasier(xml, singular):
        return {item["number"]: item["students"]
                for child in xml.findall(singular)
                for item in [assignmentFromXmlEasier(child)]}

def gradesFromXmlEasier(xml):
        return {"psets": categoryFromXmlEasier(xml.find("psets"), "pset"),
                "quizzes": categoryFromXmlEasier(xml.find("quizzes"), "quiz")}

def parseGradesEasier(filename):
        return gradesFromXmlEasier(ET.parse(filename).getroot())

In [ ]:
parseGradesEasier('grades.xml')

# Outputting Online Grades Data

In [ ]:
def splitName(name):
    raise NotImplementedError

In [ ]:
def averageToLetter(n):
    if n >= 90:
        return "A"
    elif n >= 80:
        return "B"
    elif n >= 70:
        return "C"
    elif n >= 60:
        return "D"
    else:
        return "A+"

In [ ]:
def studentAverage(email, grades):
    total = 0

    for pset in grades["psets"].values():
        if email in pset:
            total += pset[email]

    for quiz in grades["quizzes"].values():
        if email in quiz:
            total += quiz[email]

    return total / (len(grades["psets"]) + len(grades["quizzes"]))

def studentGrade(email, grades):
    return averageToLetter(studentAverage(email, grades))

In [ ]:
def outputGrades(filename, students, grades):
    with open(filename, "w") as f:
        f.write("Last Name,First Name,Middle,MIT ID,Subject #,Section #,Grade,Units,Comment\n")

        for student in students:
            last, first, middle = splitName(student["Student Name"])
            f.write(last + "," + first + "," + middle + "," \
                    + student["MIT ID"] + "," \
                    + student["Enrolled"] + "," \
                    + student["Sec"] + "," \
                    + studentGrade(student["Email Address"], grades) + "," \
                    + student["Un"] + "," \
                    + "\n")

# Reading Multiple Input Files

In [ ]:
import os

def readCsvFile(filename):
    """Read a whole CSV file that begins with a header line.
    Returns a list of dictionaries, with keys corresponding to header texts."""
    
    raise NotImplementedError

def readMultiFile(dirname):
    """Read all CSV files in a directory, to produce a grades database
    of the kind we built above."""
    
    out = {"psets": {},
           "quizzes": {}}

    for filename in os.listdir(dirname):
        lines = readCsvFile(os.path.join(dirname, filename))
        mapping = {line["Email Address"]: int(line["Grade"]) for line in lines}

        m = re.search("pset([0-9]*)\\.csv", filename)
        if m:
            out["psets"][int(m.group(1))] = mapping
        else:
            m = re.search("quiz([0-9]*)\\.csv", filename)
            if m:
                out["quizzes"][int(m.group(1))] = mapping

    return out

# Writing Multiple Output Files

In [ ]:
def makeStudentReports(dirname, students, grades):
    """Given values representing student data and grades, create the specified directory
    and populate it with one textual grade report per student."""
    os.mkdir(dirname)

    for student in students:
        with open(os.path.join(dirname, student["MIT ID"] + ".txt"), "w") as f:
            f.write("Dear " + student["Student Name"] + ",\n")
            f.write("\n")
            f.write("How are you enjoying Course " + student["Course"] + "?  Here are your grades in 6.666.\n")
            f.write("\n")

            f.write("Psets:\n")
            for pset, scores in grades["psets"].items():
                if student["Email Address"] in scores:
                    f.write("#" + str(pset) + ": " + str(scores[student["Email Address"]]) + "\n")
            f.write("\n")

            f.write("Quizzes:\n")
            for quiz, scores in grades["quizzes"].items():
                if student["Email Address"] in scores:
                    f.write("#" + str(quiz) + ": " + str(scores[student["Email Address"]]) + "\n")